## RAG NOTEBOOK:
This notebook contains the steps and code to demonstrate support of Retrieval Augumented Generation in watsonx.ai. It introduces commands for data retrieval, knowledge base building & querying, and model testing.

Some familiarity with Python is helpful.

### About Retrieval Augmented Generation
Retrieval Augmented Generation (RAG) is a versatile pattern that can unlock a number of use cases requiring factual recall of information, such as querying a knowledge base in natural language.

In its simplest form, RAG requires 3 steps:

- Index knowledge base passages (once)
- Retrieve relevant passage(s) from knowledge base (for every user query)
- Generate a response by feeding retrieved passage into a large language model (for every user query)

## Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Document data loading](#data)
- [Build up knowledge base](#build_base)
- [Foundation Models on watsonx](#models)
- [Generate a retrieval-augmented response to a question](#predict)
- [Summary and next steps](#summary)


<a id="setup"></a>
##  Set up the environment

In [1]:
!pip3 install pypdf
!pip3 install langchain-openai
!pip3 install langchain
!pip3 install chromadb


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached jiter-0.8.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (5.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.3/414.3 kB 3.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.2/473.2 kB 8.6 MB/s eta 0:00:00ta 0:00:01
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached jiter-0.8.2-cp311-cp311-macosx_11_0_arm64.whl (311 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.34
    Uninstalling langchain-core-0.3.34:
      Successfully uninstalled langchain-core-0.3.34

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To 

In [2]:
 # Import PdfReader from pypdf to read PDF files
from pypdf import PdfReader

# Load the PDF file
reader = PdfReader("IBM_Annual_Report_2023.pdf")

# Extract text from each page and strip any leading/trailing spaces
pdf_texts = [p.extract_text().strip() for p in reader.pages]

# Filter out empty strings to remove blank pages or pages with no extractable text
pdf_texts = [text for text in pdf_texts if text]

# Print the text of the 9th page (index 8)
print(pdf_texts[8])



MANAGEMENT DISCUSSION SNAPSHOT
($ and shares in millions except per share amounts)
For year ended December 31: 2023 2022 (1)
Yr.-to-Yr. 
Percent/Margin 
Change
Revenue (2) $ 61,860 $ 60,530  2.2 % 
Gross profit margin  55.4 %  54.0 %  1.4 pts. 
Total expense and other (income) $ 25,610 $ 31,531  (18.8) %    
Income from continuing operations before income taxes $ 8,690 $ 1,156  NM 
Provision for/(benefit from) income taxes from continuing operations $ 1,176 $ (626)  NM 
Income from continuing operations $ 7,514 $ 1,783  NM  
Income from continuing operations margin  12.1 %  2.9 %  9.2 pts. 
Loss from discontinued operations, net of tax $ (12) $ (143)  (91.8) %    
Net income $ 7,502 $ 1,639  NM 
Earnings per share from continuing operations–assuming dilution $ 8.15 $ 1.95  NM 
Consolidated earnings per share–assuming dilution $ 8.14 $ 1.80  NM 
Weighted-average shares outstanding–assuming dilution  922.1  912.3  1.1 % 
Assets (3)
$ 135,241 $ 127,243  6.3 %    
Liabilities (3)
$ 112,628

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [4]:
# Import RecursiveCharacterTextSplitter for text chunking
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the text splitter with custom separators and chunk size
character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],  # Define separators for splitting text (paragraphs, new lines, sentences, words, and characters)
    chunk_size=1000,  # Set the maximum size of each chunk
    chunk_overlap=0    # Set the overlap between chunks (0 means no overlap)
)

# Join all extracted PDF text with double newlines and split it into smaller chunks
character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))

# Print the 11th chunk (index 10) of the split text
print(character_split_texts[10])

# Print the total number of chunks created after splitting
print(f"\nTotal chunks: {len(character_split_texts)}")


of several critical technologies, including AI, quantum 
computing, and semiconductors. 
In AI, we demonstrated our ability to quickly transform 
research into commercial applications. We launched the 
watsonx AI and data platform, introduced the groundbreaking 
Granite AI foundational model, and developed new AI-
optimized hardware. 
We have IBM Quantum System One engagements with several 
leading organizations, including Cleveland Clinic, the Platform 
for Digital and Quantum Innovation of Quebec, Rensselaer 
Polytechnic Institute, and the University of Tokyo. We also 
IBM 2023 Annual Report 3

Total chunks: 557


In [5]:
# Import the ChromaDB library to work with vector databases
import chromadb

# Import the SentenceTransformerEmbeddingFunction utility from ChromaDB
# This function helps generate embeddings for text data
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

# Initialize an instance of the SentenceTransformer embedding function
# This function will be used to convert text into numerical vector representations (embeddings)
embedding_function = SentenceTransformerEmbeddingFunction()

# Generate the embedding for the 10th document in the 'character_split_texts' list
# and print the resulting embedding vector
print(embedding_function([character_split_texts[10]]))


/Users/suyashgupte/Documents/GitHub/opex-apis/opex/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[array([-7.89173990e-02, -4.01439816e-02, -5.06751938e-03, -4.50713672e-02,
       -6.05559945e-02, -5.35961054e-02, -6.19608238e-02,  1.74064599e-02,
       -2.08196677e-02,  3.05104759e-02, -7.06966594e-02, -3.19342948e-02,
        3.21453847e-02,  3.13325301e-02,  6.07728492e-03,  6.91314936e-02,
        6.26401007e-02, -7.42840096e-02,  4.58352687e-03, -9.77577865e-02,
       -6.77745440e-04, -1.10710936e-03,  1.70304421e-02, -4.96980734e-02,
        9.25213471e-03,  1.00498505e-01,  1.35068800e-02, -7.54344612e-02,
        1.82290673e-02, -2.45860573e-02,  1.96209382e-02,  1.38088688e-03,
        4.37018368e-03, -1.90817062e-02, -1.90199669e-02,  1.43828755e-02,
        3.03840302e-02, -9.09088776e-02,  3.65068540e-02, -5.74920848e-02,
       -2.15360653e-02, -2.52513383e-02, -7.55398050e-02,  5.72257340e-02,
        7.91857690e-02,  8.05481970e-02,  6.50972314e-03, -2.11736653e-02,
        4.14696485e-02, -7.92839900e-02, -7.32430816e-02, -7.28527457e-02,
        5.41954562e-02, 

In [6]:
# Initialize a ChromaDB client instance to interact with the database
chroma_client = chromadb.Client()

# Create a new collection in ChromaDB named "IBM_Annual_report_2023"
# The collection will store embedded documents, using the specified embedding function
chroma_collection = chroma_client.create_collection("IBM_Annual_report_2023", embedding_function=embedding_function)

# Generate unique string IDs for each document by converting their indices to strings
ids = [str(i) for i in range(len(character_split_texts))]

# Add the documents to the ChromaDB collection along with their corresponding IDs
chroma_collection.add(ids=ids, documents=character_split_texts)

# Count the number of documents stored in the collection and return the count
chroma_collection.count()


557

In [7]:
# Define the query we want to search for
query = "What was the total revenue?"

# Perform a similarity search using ChromaDB, retrieving the top 5 most relevant documents
results = chroma_collection.query(query_texts=[query], n_results=5)

# Extract the list of retrieved documents from the query results
retrieved_documents = results['documents'][0]

# Iterate through each retrieved document
for document in retrieved_documents:
    # Print the document content
    print(document)
    # Print a newline for better readability between documents
    print('\n')


Revenue Recognized for Performance Obligations Satisfied (or Partially Satisfied) in Prior Periods
For the year ended December  31, 2023, revenue was reduced by $16 million for performance obligations satisfied or partially 
satisfied in previous periods mainly due to changes in estimates on contracts with cost-to-cost measures of progress. Refer to note 
A, “Significant Accounting Policies,” for additional information on these contracts and estimates of costs to complete.
Reconciliation of Contract Balances
The following table provides information about notes and accounts receivable—trade, contract assets and deferred income 
balances.
($ in millions)
At December 31: 2023 2022
Notes and accounts receivable — trade (net of allowances of $192 in 2023 and $233 in 2022) $ 7,214 $ 6,541 
Contract assets (1)  505  464 
Deferred income (current)  13,451  12,032 
Deferred income (noncurrent)  3,533  3,499


Total revenue $ 61,860 $ 60,530  2.2 %  2.9 %
Total gross profit $ 34,300 $ 32,687  4.

In [8]:
def retreiver(query):
  # Perform a similarity search using ChromaDB, retrieving the top 5 most relevant documents
  results = chroma_collection.query(query_texts=[query], n_results=5)

  # Extract the list of retrieved documents from the query results
  retrieved_documents = results['documents'][0]
  return retrieved_documents

In [16]:
openai_api_key = ""


In [27]:
from langchain_openai import OpenAI

llm = OpenAI(
    model="gpt-4o-mini",
    temperature=0,
    api_key=openai_api_key
)

In [28]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
        """
        |System|
        You are a financial research analyst. You have to analyse the Request For Proposal(RFP) documents for the bidding process.

        |Instruction|
        Refer to the context from RFP document and answer the following question.Answer it in a concise manner. Do not add any additonal information.

        |Question|
        {question}

        |Context|
        {context}

        |Answer|
        """
        )




In [29]:
query1="What was the total revenue?"
documents1=retreiver(query1)

In [30]:
prompt1=prompt_template.format(question=query1, context=documents1)

In [31]:
print(llm.invoke(prompt1))

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [18]:
query2="What are IBM’s top-performing geographic regions in terms of revenue?"
documents2=retreiver(query2)

In [19]:
prompt2=prompt_template.format(question=query2, context=documents2)

In [20]:
print(llm.invoke(prompt2))


        The United States is IBM's top-performing geographic region in terms of revenue.


In [21]:
query3="How is IBM reducing its carbon footprint?"
documents3=retreiver(query3)

In [22]:
prompt3=prompt_template.format(question=query3, context=documents3)

In [23]:
print(llm.invoke(prompt3))


        IBM is reducing its carbon footprint by achieving a 63% reduction in greenhouse gas emissions against the base year 2010.
